In [ ]:
import zipfile
import os

In [ ]:
!wget --no-check-certificate \
    "https://github.com/m-sumaim/corn_seed_dataset/archive/refs/heads/main.zip" \
    -O "/tmp/seed_class.zip"


zip_ref = zipfile.ZipFile('/tmp/seed_class.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-11-20 20:15:45--  https://github.com/m-sumaim/corn_seed_dataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/m-sumaim/corn_seed_dataset/zip/refs/heads/main [following]
--2023-11-20 20:15:45--  https://codeload.github.com/m-sumaim/corn_seed_dataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/seed_class.zip’

/tmp/seed_class.zip     [            <=>     ] 491.66M  44.3MB/s    in 11s     

2023-11-20 20:15:56 (44.3 MB/s) - ‘/tmp/seed_class.zip’ saved [515546839]



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
#from PIL import Image as image
import numpy as np

# Set the directories for the training and testing data
train_dir = '/tmp/corn_seed_dataset-main/train'
test_dir = '/tmp/corn_seed_dataset-main/validation'

# Create an ImageDataGenerator for the training data with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create an ImageDataGenerator for the testing data without data augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Create a train generator that generates batches of augmented images from the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Create a test generator that generates batches of images from the testing data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=test_generator)

# Save the trained model
model.save('corn_seed_classifier.h5')

# Load the trained model
model = tf.keras.models.load_model('corn_seed_classifier.h5')

from google.colab import files

files.download("corn_seed_classifier.h5")

Found 14322 images belonging to 4 classes.
Found 3474 images belonging to 4 classes.
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
448/448 [==============================] - 172s 337ms/step - loss: 1.1952 - accuracy: 0.4568 - val_loss: 1.0508 - val_accuracy: 0.5403
Epoch 2/50
448/448 [==============================] - 146s 326ms/step - loss: 1.0366 - accuracy: 0.5546 - val_loss: 0.9463 - val_accuracy: 0.6002
Epoch 3/50
448/448 [==============================] - 150s 334ms/step - loss: 0.9192 - accuracy: 0.6214 - val_loss: 0.8523 - val_accuracy: 0.6382
Epoch 4/50
448/448 [==============================] - 144s 321ms/step - loss: 0.8411 - accuracy: 0.6591 - val_loss: 0.7493 - val_accuracy: 0.6952
Epoch 5/50
448/448 [==============================] - 145s 324ms/step - loss: 0.7823 - accuracy: 0.6895 - val_loss: 0.6895 - val_accuracy: 0.7182
Epoch 6/50
448/448 [==============================] - 146s 326ms/step - loss: 0.7357 - accuracy: 0.7099 - val_loss: 0.65

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>